# Module 8: Regression Metrics

**Goal:** Understand when to use MAE vs RMSE vs R², interpret residual plots, and communicate model performance.

**Prerequisites:** Module 3 (Linear Regression)

**Expected Runtime:** ~20 minutes

**Outputs:**
- Side-by-side metric comparisons
- Residual analysis plots
- Business-ready performance summary

---

## Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## Part 1: Generate Sample Data

We'll create a synthetic LTV prediction problem with known characteristics.

In [ ]:
# Generate synthetic customer LTV data
n_samples = 500

# Features
tenure_months = np.random.uniform(1, 48, n_samples)
avg_order_value = np.random.uniform(20, 200, n_samples)
orders_per_month = np.random.uniform(0.5, 4, n_samples)

# True LTV relationship (with some noise)
true_ltv = (
    50 +  # base value
    tenure_months * 5 +  # loyalty effect
    avg_order_value * 2 +  # spending effect
    orders_per_month * 30 +  # frequency effect
    np.random.normal(0, 50, n_samples)  # noise
)

# Create DataFrame
df = pd.DataFrame({
    'tenure_months': tenure_months,
    'avg_order_value': avg_order_value,
    'orders_per_month': orders_per_month,
    'ltv': true_ltv
})

# Add a few outliers (high-value customers)
outlier_idx = np.random.choice(n_samples, size=10, replace=False)
df.loc[outlier_idx, 'ltv'] += np.random.uniform(300, 600, 10)

print("Dataset shape:", df.shape)
df.describe()

## Part 2: Train a Simple Model

In [ ]:
# Split data
X = df[['tenure_months', 'avg_order_value', 'orders_per_month']]
y = df['ltv']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predictions
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

print(f"Training samples: {len(X_train)}")
print(f"Test samples: {len(X_test)}")

## Part 3: Calculate All Metrics

In [ ]:
def calculate_all_metrics(y_true, y_pred, name=""):
    """Calculate and display all regression metrics."""
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    # MAPE (avoiding division by zero)
    mask = y_true != 0
    mape = np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100
    
    print(f"\n=== {name} Metrics ===")
    print(f"MAE:  ${mae:.2f}  (average absolute error)")
    print(f"RMSE: ${rmse:.2f}  (penalizes big errors)")
    print(f"R²:   {r2:.4f}  ({r2*100:.1f}% variance explained)")
    print(f"MAPE: {mape:.1f}%  (percentage error)")
    print(f"\nRMSE/MAE ratio: {rmse/mae:.2f}")
    if rmse/mae > 1.3:
        print("⚠️  Ratio > 1.3 suggests outlier errors")
    
    return {'mae': mae, 'rmse': rmse, 'r2': r2, 'mape': mape}

train_metrics = calculate_all_metrics(y_train, y_pred_train, "Training Set")
test_metrics = calculate_all_metrics(y_test, y_pred_test, "Test Set")

## Part 4: Residual Analysis

Residuals = Actual - Predicted. They reveal *how* your model is wrong.

In [ ]:
# Calculate residuals
residuals_test = y_test - y_pred_test

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Predicted vs Actual
ax1 = axes[0, 0]
ax1.scatter(y_pred_test, y_test, alpha=0.5, c='#0ea5e9')
ax1.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'g--', lw=2, label='Perfect predictions')
ax1.set_xlabel('Predicted LTV ($)')
ax1.set_ylabel('Actual LTV ($)')
ax1.set_title('Predicted vs Actual')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Residuals vs Predicted
ax2 = axes[0, 1]
ax2.scatter(y_pred_test, residuals_test, alpha=0.5, c='#0ea5e9')
ax2.axhline(y=0, color='r', linestyle='--', lw=2)
ax2.set_xlabel('Predicted LTV ($)')
ax2.set_ylabel('Residual ($)')
ax2.set_title('Residuals vs Predicted (look for patterns!)')
ax2.grid(True, alpha=0.3)

# 3. Residual Distribution
ax3 = axes[1, 0]
ax3.hist(residuals_test, bins=30, color='#0ea5e9', edgecolor='white', alpha=0.7)
ax3.axvline(x=0, color='r', linestyle='--', lw=2, label='Zero (ideal center)')
ax3.axvline(x=residuals_test.mean(), color='orange', linestyle='-', lw=2, label=f'Mean: ${residuals_test.mean():.1f}')
ax3.set_xlabel('Residual ($)')
ax3.set_ylabel('Count')
ax3.set_title('Residual Distribution (should be centered at 0)')
ax3.legend()

# 4. Q-Q Plot (normality check)
ax4 = axes[1, 1]
from scipy import stats
stats.probplot(residuals_test, dist="norm", plot=ax4)
ax4.set_title('Q-Q Plot (should follow diagonal for normal errors)')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Interpretation
print("\n📊 Residual Analysis Summary:")
print(f"  Mean residual: ${residuals_test.mean():.2f} (should be ~0)")
print(f"  Std of residuals: ${residuals_test.std():.2f}")
print(f"  Skewness: {residuals_test.skew():.2f} (0 = symmetric)")

## Part 5: Comparing to Baseline

Always compare your model to a naive baseline. The simplest: predict the mean.

In [ ]:
# Baseline: predict the training mean for everyone
baseline_pred = np.full(len(y_test), y_train.mean())

baseline_mae = mean_absolute_error(y_test, baseline_pred)
baseline_rmse = np.sqrt(mean_squared_error(y_test, baseline_pred))

print("=== Baseline (Predict Mean) ===")
print(f"Prediction: ${y_train.mean():.2f} for everyone")
print(f"Baseline MAE:  ${baseline_mae:.2f}")
print(f"Baseline RMSE: ${baseline_rmse:.2f}")

print("\n=== Our Model ===")
print(f"Model MAE:  ${test_metrics['mae']:.2f}")
print(f"Model RMSE: ${test_metrics['rmse']:.2f}")

print("\n=== Improvement ===")
mae_improvement = (baseline_mae - test_metrics['mae']) / baseline_mae * 100
rmse_improvement = (baseline_rmse - test_metrics['rmse']) / baseline_rmse * 100
print(f"MAE improvement:  {mae_improvement:.1f}% reduction")
print(f"RMSE improvement: {rmse_improvement:.1f}% reduction")

## Part 6: Impact of Outliers

Let's see how outliers affect MAE vs RMSE differently.

In [ ]:
# Identify outlier errors
error_threshold = 2 * residuals_test.std()  # ~95th percentile
outlier_mask = np.abs(residuals_test) > error_threshold

print(f"Error threshold: ${error_threshold:.2f}")
print(f"Outlier predictions: {outlier_mask.sum()} ({outlier_mask.sum()/len(y_test)*100:.1f}%)")

# Metrics with and without outliers
y_test_clean = y_test[~outlier_mask]
y_pred_clean = y_pred_test[~outlier_mask]

mae_all = mean_absolute_error(y_test, y_pred_test)
rmse_all = np.sqrt(mean_squared_error(y_test, y_pred_test))

mae_clean = mean_absolute_error(y_test_clean, y_pred_clean)
rmse_clean = np.sqrt(mean_squared_error(y_test_clean, y_pred_clean))

print("\n         |  All Data  |  Without Outliers  |  Change")
print("-" * 55)
print(f"MAE      |  ${mae_all:>7.2f}  |  ${mae_clean:>7.2f}          |  {(mae_all-mae_clean)/mae_all*100:>+5.1f}%")
print(f"RMSE     |  ${rmse_all:>7.2f}  |  ${rmse_clean:>7.2f}          |  {(rmse_all-rmse_clean)/rmse_all*100:>+5.1f}%")
print(f"RMSE/MAE |  {rmse_all/mae_all:>8.2f}  |  {rmse_clean/mae_clean:>8.2f}          |")

print("\n💡 Notice: RMSE is more affected by outliers than MAE!")

## Part 7: TODO - Segmented Analysis

Overall metrics can hide problems with specific customer segments.

In [ ]:
# Analyze metrics by customer segment
def segment_analysis(y_true, y_pred, segment_col, segment_name):
    """Analyze metrics by segment."""
    results = []
    for segment in sorted(segment_col.unique()):
        mask = segment_col == segment
        if mask.sum() > 5:
            mae = mean_absolute_error(y_true[mask], y_pred[mask])
            rmse = np.sqrt(mean_squared_error(y_true[mask], y_pred[mask]))
            r2 = r2_score(y_true[mask], y_pred[mask])
            results.append({
                segment_name: segment,
                'Count': mask.sum(),
                'MAE': mae,
                'RMSE': rmse,
                'R²': r2
            })
    return pd.DataFrame(results)

# Create LTV segments and analyze performance in each
ltv_segments = pd.qcut(y_test, q=3, labels=['Low', 'Medium', 'High'])
segment_results = segment_analysis(y_test.values, y_pred_test, ltv_segments, 'LTV Segment')

print("\n=== Performance by LTV Segment ===")
print(segment_results.to_string(index=False))

# Insight: Which segment has the worst R²? That's where the model struggles most.

## Part 8: Stakeholder Summary

### TODO: Write a 3-bullet summary (~100 words) for stakeholders

Template:
• **Model accuracy:** Average prediction error is $____ (MAE). The model explains ____% of LTV variance (R²).
• **Segment performance:** The model performs [best/worst] for [segment] customers because [reason].
• **Business impact:** [What does this accuracy mean for decisions? Where should we not trust the model?]

### Your Summary:

*Write your stakeholder summary here...*

---

## Part 9: Visualization for Stakeholders

In [ ]:
# Create a stakeholder-friendly visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Prediction accuracy by value range
ax1 = axes[0]
bins = [0, 200, 400, 600, 800, 1000, float('inf')]
labels = ['$0-200', '$200-400', '$400-600', '$600-800', '$800-1000', '$1000+']
y_binned = pd.cut(y_test, bins=bins, labels=labels)

y_pred_series = pd.Series(y_pred_test, index=y_test.index)
segment_mae = y_test.groupby(y_binned).apply(lambda x: mean_absolute_error(x, y_pred_series[x.index]))
segment_counts = y_binned.value_counts().sort_index()

ax1.bar(range(len(segment_mae)), segment_mae.values, color='#0ea5e9', alpha=0.7)
ax1.set_xticks(range(len(labels)))
ax1.set_xticklabels(labels, rotation=45, ha='right')
ax1.set_ylabel('Mean Absolute Error ($)')
ax1.set_title('Prediction Error by Customer Value')
ax1.grid(True, alpha=0.3, axis='y')

# Right: Error distribution in business terms
ax2 = axes[1]
abs_errors = np.abs(residuals_test)
thresholds = [25, 50, 100, 200]
percentages = [(abs_errors <= t).mean() * 100 for t in thresholds]

ax2.barh(range(len(thresholds)), percentages, color='#22c55e', alpha=0.7)
ax2.set_yticks(range(len(thresholds)))
ax2.set_yticklabels([f'Within ${t}' for t in thresholds])
ax2.set_xlabel('% of Predictions')
ax2.set_title('Prediction Accuracy Breakdown')
ax2.set_xlim(0, 100)

for i, pct in enumerate(percentages):
    ax2.text(pct + 2, i, f'{pct:.0f}%', va='center')

plt.tight_layout()
plt.show()

print("\n📈 Key Takeaway for Stakeholders:")
print(f"  • {percentages[1]:.0f}% of predictions are within $50 of actual LTV")
print(f"  • Average prediction error: ${test_metrics['mae']:.0f}")
print(f"  • Model captures {test_metrics['r2']*100:.0f}% of what drives customer value")

## Self-Check

Uncomment and run the asserts below to verify your regression metrics are computed correctly.

In [ ]:
# SELF-CHECK: Verify your regression metrics
assert test_metrics['mae'] > 0 and test_metrics['mae'] < 500, f"MAE should be reasonable for LTV, got {test_metrics['mae']:.1f}"
assert test_metrics['rmse'] >= test_metrics['mae'], "RMSE should be >= MAE (penalizes large errors more)"
assert -0.5 <= test_metrics['r2'] <= 1.0, f"R² should be in [-0.5, 1.0], got {test_metrics['r2']:.3f}"
assert test_metrics['mae'] < baseline_mae, f"Model MAE ({test_metrics['mae']:.1f}) should beat baseline ({baseline_mae:.1f})"
print(f"✅ Self-check passed! MAE: {test_metrics['mae']:.1f}, RMSE: {test_metrics['rmse']:.1f}, R²: {test_metrics['r2']:.3f}")

## Key Takeaways

1. **MAE** tells you average error in business units - great for communication
2. **RMSE** penalizes big errors more - use when large mistakes are costly
3. **R²** compares to baseline - always pair with error metrics
4. **Residual plots** reveal patterns that overall metrics hide
5. **Segment analysis** exposes where the model struggles

### Next Steps
- Explore the interactive playground to see how different error patterns affect metrics
- Complete the quiz to test your understanding